In [ ]:
!pip install transformers datasets accelerate evaluate torch torchvision

In [ ]:
!pip install --upgrade transformers datasets

In [ ]:
import os
import shutil
import requests

cache_dir = os.path.expanduser("~/.cache/huggingface/datasets")
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)

current_dir_cache = "./.cache"
if os.path.exists(current_dir_cache):
    print(f"현재 디렉토리 캐시 폴더 삭제: {current_dir_cache}")
    shutil.rmtree(current_dir_cache)

# 다른 필수 라이브러리 설치
!pip install transformers datasets accelerate evaluate torch torchvision scikit-learn

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Hugging Face datasets 캐시 및 임시 캐시 폴더 확인 및 삭제 중...
Hugging Face datasets 캐시 폴더 없음. 건너뜀.
현재 디렉토리 캐시 폴더 없음. 건너뜀.

필수 라이브러리 완전 삭제 및 재설치 중...
Found existing installation: requests 2.32.4
Uninstalling requests-2.32.4:
  Successfully uninstalled requests-2.32.4
Found existing installation: urllib3 1.26.18
Uninstalling urllib3-1.26.18:
  Successfully uninstalled urllib3-1.26.18
Found existing installation: idna 3.7
Uninstalling idna-3.7:
  Successfully uninstalled idna-3.7
requests, urllib3, idna 제거 완료 (존재하는 경우).
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

requests, urllib3, idna 특정 버전 설치 완료.
ERROR: Operation cancelled by user
^C


In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import os
import shutil
import torch
import pandas as pd

model_name = "google/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name)

# 모델의 레이블 정보 확인
print(f"모델의 클래스 수: {len(model.config.id2label)}")
print(f"모델의 첫 5개 클래스: {list(model.config.id2label.values())[:5]}...")

# classify_and_organize_food_images
def classify_and_organize_food_images(image_folder_path, output_base_path):

    # 출력 기본 경로가 없으면 생성
    if not os.path.exists(output_base_path):
        os.makedirs(output_base_path)

    # GPU 사용 가능 여부 확인
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # 지정된 이미지 폴더 내의 모든 파일 순회
    for filename in os.listdir(image_folder_path):
        # 이미지 파일인지 확인 (일반적인 이미지 확장자)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            image_full_path = os.path.join(image_folder_path, filename)

            try:
                # 이미지 로드 및 RGB로 변환
                image = Image.open(image_full_path).convert("RGB")

                # 이미지 전처리 및 모델 예측
                inputs = processor(images=image, return_tensors="pt")
                inputs = {k: v.to(device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = model(**inputs)
                logits = outputs.logits

                predicted_class_idx = logits.argmax(-1).item()
                predicted_label = model.config.id2label[predicted_class_idx]

                output_food_folder = os.path.join(output_base_path, predicted_label)
                if not os.path.exists(output_food_folder):
                    os.makedirs(output_food_folder)

                # 원본 이미지를 예측된 음식 종류 폴더로 복사
                shutil.copy(image_full_path, os.path.join(output_food_folder, filename))

            except Exception as e:
                print(f"오류 발생 ({filename}): {e}")

my_image_folder = "/content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example" # 경로 수정하기
output_food_classified_folder = "/content/drive/MyDrive/Food101xOutput" # 경로 수정하기
classify_and_organize_food_images(my_image_folder, output_food_classified_folder)

def export_classified_images_to_csv(output_base_path, export_path_csv="classified_results.csv", export_path_xlsx=None):
    records = []

    for label in os.listdir(output_base_path):
        class_folder = os.path.join(output_base_path, label)
        if os.path.isdir(class_folder):
            for image_file in os.listdir(class_folder):
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    full_path = os.path.join(class_folder, image_file)
                    records.append({
                        "image_filename": image_file,
                        "predicted_label": label,
                        "full_path": full_path
                    })

    if records:
        df = pd.DataFrame(records)
        df.to_csv(export_path_csv, index=False)
        print(f"CSV 파일 저장 완료: {export_path_csv}")

        if export_path_xlsx:
            try:
                df.to_excel(export_path_xlsx, index=False)
            except ImportError:
                print("XLSX 파일 저장 실패")
    else:
        print("XLSX 파일 저장 실패")

export_classified_images_to_csv(
    output_base_path=output_food_classified_folder,
    export_path_csv="/content/drive/MyDrive/food101x_classified_summary.csv", # Drive에 저장될 CSV 파일
    export_path_xlsx="/content/drive/MyDrive/food101x_classified_summary.xlsx" # Drive에 저장될 XLSX 파일
)
print("완료")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

사전 학습된 'google/vit-base-patch16-224' 모델과 프로세서 로드 완료!
모델의 클래스 수: 1000
모델의 첫 5개 클래스: ['tench, Tinca tinca', 'goldfish, Carassius auratus', 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias', 'tiger shark, Galeocerdo cuvieri', 'hammerhead, hammerhead shark']...

음식 이미지 분류 및 정리 시작 (사전 학습된 ViT 모델 사용)...
출력 폴더 생성: /content/drive/MyDrive/Food101xOutput
이미지 처리 중: /content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example/MyFoodExample (357).jpg
  -> 'MyFoodExample (357).jpg'를 'corn' 폴더로 분류 및 복사 완료.
이미지 처리 중: /content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example/MyFoodExample (1).jpg
  -> 'MyFoodExample (1).jpg'를 'plate' 폴더로 분류 및 복사 완료.
이미지 처리 중: /content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example/MyFoodExample (2).jpg
  -> 'MyFoodExample (2).jpg'를 'hot pot, hotpot' 폴더로 분류 및 복사 완료.
이미지 처리 중: /content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example/MyFoodExample (3).jpg
  -> 'MyFoodExample (3).jpg'를 'hot pot, hotpot' 폴더로 분류 및 복사 완료.
이미지 처리 중: /con